In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

data_path = "../data/"
data = pd.read_csv(data_path + "dengue2.csv")

dengue2.csv는 Aedes 모기에 의해 전염되는 뎅기열과 관련된 자료이다. 
-	ID: subject ID
-	DENGUE: Dengue fever status: 1=yes, 0=no
-	AGE in years
-	MOSNET: Use of mosquito netting: 1=yes, 0=no 
-	SECTOR: Geographical sector in which the subject lived: 1,2,3,4, or 5


In [3]:
data.head()

,ID,AGE,SEX,MOSNET,DENGUE,SCREEN,SES,SECTOR
0,501,16,2,0,0,2,2.0,1
1,502,33,1,0,0,1,2.0,1
2,503,1,1,0,0,3,2.0,1
3,504,35,1,0,0,1,2.0,1
4,505,6,1,0,0,1,2.0,1


1. AGE, MOSNET, SECTOR를 사용하여 DENGUE를 예측하는 로지스틱 회귀모형을 적합하고 적합된 로지스틱회귀식을 쓰시오.

In [7]:
model = smf.glm('DENGUE~AGE+MOSNET+C(SECTOR)', data, family = sm.families.Binomial()).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 DENGUE   No. Observations:                  196
Model:                            GLM   Df Residuals:                      189
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -101.85
Date:                Mon, 26 Apr 2021   Deviance:                       203.71
Time:                        16:35:08   Pearson chi2:                     188.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.7866      1.038     -3.648      0.000      -5.821      -1.752
C(SECTOR)[T.2]     1.5612      1.117      1.397      0.162      -0.629       3.751
C(SECTOR)[T.3]     3.0322      1.078      2.812      0.005       0.919       5.146
C(SECTOR)[T.4]     2.7510      1.077      2.553      0.011       0.639       4.863
C(SECTOR)[T.5]     2.2200      1.072      2.070      0.038       0.118       4.322
AGE                0.0243      0.009      2.679      0.007       0.007       0.042
MOSNET            -0.3335      1.272     -0.262      0.793      -2.826       2.159
==================================================================================
"""

log(P(X)/(1-P(X))) = -3.79 + (1.56*sector2) + (3.03*sector3) + (2.75*sector4) + (2.22*sector5) + (0.02*AGE) - (0.33*MOSNET)



2. 각 회귀계수가 유의한지 검정하시오.

- SECTOR[T.2]와 MOSNET은 유의하지 않다.

3. 나이와 sector가 동일하다면 모기망을 사용하지 않을 때에 비해 사용할 때 뎅기열에 전염될 odds는 몇 % 감소하는가? 

In [8]:
np.exp(-0.3335)

0.7164118986399645

In [9]:
1-np.exp(-0.3335) # 28.4% 감소

0.28358810136003554

3-1. 나이와 sector가 동일하다면 모기망을 사용할 때에 비해 사용하지 않을 때 뎅기열에 전염될 odds는 몇 % 증가하는가?

In [14]:
data['MOSNET_reverse'] = 1-data.MOSNET

In [15]:
model = smf.glm('DENGUE~AGE+MOSNET_reverse+C(SECTOR)', data, family = sm.families.Binomial()).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 DENGUE   No. Observations:                  196
Model:                            GLM   Df Residuals:                      189
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -101.85
Date:                Mon, 26 Apr 2021   Deviance:                       203.71
Time:                        16:50:40   Pearson chi2:                     188.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -4.1201      1.598     -2.578      0.010      -7.252      -0.988
C(SECTOR)[T.2]     1.5612      1.117      1.397      0.162      -0.629       3.751
C(SECTOR)[T.3]     3.0322      1.078      2.812      0.005       0.919       5.146
C(SECTOR)[T.4]     2.7510      1.077      2.553      0.011       0.639       4.863
C(SECTOR)[T.5]     2.2200      1.072      2.070      0.038       0.118       4.322
AGE                0.0243      0.009      2.679      0.007       0.007       0.042
MOSNET_reverse     0.3335      1.272      0.262      0.793      -2.159       2.826
==================================================================================
"""

In [13]:
np.exp(0.3335) # 39.6% 증가, 값은 동일, 부호만 변경

1.39584504654152

4. 다른 설명변수의 값이 동일하다면 나이가 1살 증가하면 뎅기열에 걸릴 odds가 몇 % 증가하는가? 5살 증가하면 몇 % 증가하는가? 

In [10]:
np.exp(0.0243) # 1살 증가시, 2.4% 증가

1.0245976510836627

In [11]:
np.exp(0.0243*5) # 5살 증가시 12.9% 증가

1.1291893659251577

5. 다른 설명변수의 값이 동일할 때 가장 뎅기열에 걸릴 위험이 높은 sector는 어디인가? 그곳의 위험은 sector 1에 비해 몇 배 높은가?

- sector3이 제일 위험, sector1이 제일 안전

In [12]:
np.exp(3.0322) # 20.7배 위험하다.

20.742816625139927